In [7]:
import numpy as np
import csv
import random
import math
import matplotlib

import matplotlib.pyplot as plt

%matplotlib inline

In [8]:
from clease.tools import update_db
from clease import Concentration
from clease import CEBulk
from clease import Evaluate
from clease import NewStructures
from clease.calculator import Clease
from clease.calculator import attach_calculator

In [9]:
import json
with open('agpd_dft_eci_ref_duplication_1_18.json') as json_file: 
    eci = json.load(json_file) 
    
conc = Concentration(basis_elements=[['Ag', 'Pd']])
settings = CEBulk(crystalstructure='fcc',
                   a=4.09,
                   size=[3,3,3],
                   concentration=conc,
                   db_name="agpd.db",
                   max_cluster_size=4,
                   max_cluster_dia=[8.0, 6.5, 5.5])
atoms = settings.atoms.copy()
atoms = attach_calculator(settings, atoms=atoms, eci=eci)

In [10]:
from clease.montecarlo.observers import ConcentrationObserver
import copy

In [11]:
from clease.montecarlo import SGCMonteCarlo
import time

In [12]:
print(eci)

{'c0': -0.044690785812717196, 'c1_0': -0.036550894703191145, 'c2_d0000_0_00': 0.027999774592412596, 'c2_d0001_0_00': 0.012756278103062423, 'c2_d0002_0_00': 0.016272362601737284, 'c2_d0003_0_00': -0.00012709442165610707, 'c2_d0004_0_00': -0.00021867027237829073, 'c2_d0005_0_00': 0.002706525627772332, 'c2_d0006_0_00': -0.003937914296133244, 'c3_d0000_0_000': 0.01080659385192681, 'c3_d0001_0_000': 0.0002583486959874537, 'c3_d0002_0_000': 0.005009506051704027, 'c3_d0003_0_000': 0.0003345994746677683, 'c3_d0003_1_000': 0.003964559117400637, 'c3_d0003_2_000': 0.0005653006830542463, 'c3_d0004_0_000': -0.0007738774287809891, 'c3_d0004_1_000': 0.0034652911938237822, 'c3_d0005_0_000': 0.00091396918756248, 'c3_d0005_1_000': 1.9546090260235544e-05, 'c3_d0006_0_000': 0.000580597715878553, 'c3_d0007_0_000': 0.0019192839101572629, 'c3_d0007_1_000': 0.003615453449340139, 'c3_d0007_2_000': 0.0017477867035675665, 'c3_d0008_0_000': -0.001670475735032817, 'c3_d0008_1_000': 0.004936115099598723, 'c3_d0009_

In [15]:

temp = np.linspace(100,800,26)
chem_p = np.linspace(-0.2,0.2,41)
sign = np.linspace(-1,1,2)

print(chem_p)
print(temp)
print(sign)
concs = np.zeros((chem_p.shape[0],2,temp.shape[0]))
times = np.zeros((chem_p.shape[0],2,temp.shape[0]))

[-0.2  -0.19 -0.18 -0.17 -0.16 -0.15 -0.14 -0.13 -0.12 -0.11 -0.1  -0.09
 -0.08 -0.07 -0.06 -0.05 -0.04 -0.03 -0.02 -0.01  0.    0.01  0.02  0.03
  0.04  0.05  0.06  0.07  0.08  0.09  0.1   0.11  0.12  0.13  0.14  0.15
  0.16  0.17  0.18  0.19  0.2 ]
[100. 128. 156. 184. 212. 240. 268. 296. 324. 352. 380. 408. 436. 464.
 492. 520. 548. 576. 604. 632. 660. 688. 716. 744. 772. 800.]
[-1.  1.]


In [25]:
temp = np.linspace(250,750,2)
chem_p = np.linspace(-0.2,0.2,41)
sign = np.linspace(-1,1,2)
concs = np.zeros((chem_p.shape[0],2,temp.shape[0]))
times = np.zeros((chem_p.shape[0],2,temp.shape[0]))
print(temp)

[250. 750.]


In [26]:
size = 27
import random
for i in range(temp.shape[0]):
    for m in range(sign.shape[0]):
        tmp = temp[i]
        eci_temp = copy.deepcopy(eci)

        atoms = attach_calculator(settings, atoms=atoms, eci=eci_temp)
        for k in range(len(atoms.numbers)):
            if np.random.rand() > 0.5:
                atoms.numbers[k] = 46
            else:
                atoms.numbers[k] = 47
        mc = SGCMonteCarlo(atoms, tmp, symbols=['Ag', 'Pd'])
        obs = ConcentrationObserver(atoms, element='Ag')
        mc.attach(obs)
        for j in range(chem_p.shape[0]):
            ch_p = chem_p[j]*sign[m]
            
            ##
            start = time.time()
            mc.run(steps = size*100,chem_pot = {'c1_0': ch_p})
            obs.reset()
            mc.run(steps=size*1000,chem_pot = {'c1_0': ch_p})
            end = time.time()
            times[j,m,i] = end-start
            concs[j,m,i] = obs.get_averages()['conc_Ag']
            print('here')

Probing energy bias using 54 MC steps...
Energy after probing: -4.468610150775254
Bias subtracted from empty cluster...
Reached maximum number of steps (2700 mc steps)
Empty cluster ECI reset to original value...
Probing energy bias using 54 MC steps...
Energy after probing: -5.052376193459876
Bias subtracted from empty cluster...
Reached maximum number of steps (27000 mc steps)
Empty cluster ECI reset to original value...
here
Probing energy bias using 54 MC steps...
Energy after probing: -4.889952563545739
Bias subtracted from empty cluster...
Reached maximum number of steps (2700 mc steps)
Empty cluster ECI reset to original value...
Probing energy bias using 54 MC steps...
Energy after probing: -4.81334833586258
Bias subtracted from empty cluster...
Reached maximum number of steps (27000 mc steps)
Empty cluster ECI reset to original value...
here
Probing energy bias using 54 MC steps...
Energy after probing: -4.63995256354574
Bias subtracted from empty cluster...
Reached maximum nu

In [29]:
np.savetxt("AgPdDFT_ref_SGC_3_concs_final_paper_np_41_6.csv", concs[:,0,:], delimiter=",")
np.savetxt("AgPdDFT_ref_SGC_3_concs_final_paper_pn_41_6.csv", concs[:,1,:], delimiter=",")